In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -r requirement.txt 

     |████████████████████████████████| 12.5MB 4.1MB/s 
     |████████████████████████████████| 952kB 34.0MB/s 
     |████████████████████████████████| 133kB 54.5MB/s 
     |████████████████████████████████| 2.7MB 27.9MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 34.2MB 211kB/s 
     |████████████████████████████████| 108.9MB 72kB/s 
     |████████████████████████████████| 6.4MB 16.4MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 2.0MB 42.9MB/s 
     |████████████████████████████████| 204kB 57.2MB/s 
     |████████████████████████████████| 194kB 45.4MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 1.9MB 31.1MB/s 
     |████████████████████████████████| 645kB 39.1MB/s 
     |████████████████████████████████| 7.2MB 42.5MB/s 
     |████████████████████████████████| 102kB 7.9MB/s 


# Library

In [4]:
import matplotlib.pyplot as plt
from PIL import Image

from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from PaddleOCR.paddleocr import PaddleOCR


import io
import cv2
import numpy as np
from PIL import Image
import sys
import pandas as pd
import fitz
from tqdm import tqdm

In [5]:
config = Cfg.load_config_from_name('vgg_seq2seq')
config['predictor']['beamsearch']=False
config['device'] = 'cpu'
config['weights'] = '/content/drive/MyDrive/ Image CVs/vietocr/transformerocr.pth'
detector = Predictor(config)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


In [6]:
import time
ocr = PaddleOCR(use_angle_cls=True, lang='en', det_model_dir='/content/drive/MyDrive/ Image CVs/PaddleOCR/en_ppocr_mobile_v2.0_det_infer', 
                rec_model_dir='/content/drive/MyDrive/ Image CVs/PaddleOCR/en_ppocr_mobile_v2.0_det_infer',
                cls_model_dir='/content/drive/MyDrive/ Image CVs/PaddleOCR/en_ppocr_mobile_v2.0_det_infer') # need to run only once to download and load model into memory

In [7]:
def img2text(img):
  
  # Input: array image
  # Output: a dataframe with bounding box and text

  # Detect line
  result = ocr.ocr(img, rec=False)
  result.reverse()

  # Get bbox
  xs = []
  ys = []
  ws = []
  hs = []
  list_text = []

  # Create df
  for line in result:
      bbox = (line[0][0]-5, line[0][1]-5, line[2][0]+5, line[2][1]+5);
      xs.append(bbox[0])
      ys.append(bbox[1])
      ws.append(bbox[2])
      hs.append(bbox[3])

      new_image = img.crop(bbox)
      list_text.append(detector.predict(new_image))
  bbox_df = pd.DataFrame({'x':xs, 'y':ys, 'w':ws, 'h':hs, 'text': list_text})
  return bbox_df

In [8]:
def img2text(img):
  # Detect line
  result = ocr.ocr(img, rec=False)
  result.reverse()

  # Get bbox
  xs = []
  ys = []
  ws = []
  hs = []
  list_text = []

  img = Image.fromarray(img)
  # Create df
  for line in result:
      bbox = (line[0][0]-5, line[0][1]-5, line[2][0]+5, line[2][1]+5);
      xs.append(bbox[0])
      ys.append(bbox[1])
      ws.append(bbox[2])
      hs.append(bbox[3])
      new_image = img.crop(bbox)
      list_text.append(detector.predict(new_image))
  bbox_df = pd.DataFrame({'x':xs, 'y':ys, 'w':ws, 'h':hs, 'text': list_text})
  return bbox_df

In [9]:
def pix2np(pix):
    if pix.n>3:
        pix = fitz.Pixmap(fitz.csRGB, pix)
    im = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    return im

In [10]:
def pdfimage2text(file):
  doc = fitz.Document(file)
  pages = doc.pageCount
  block_df = pd.DataFrame()
  for num_page in range(pages):
    pix = doc[num_page].getPixmap(alpha = False)
    pix = pix2np(pix)
    block_df = pd.concat([block_df, img2text(pix)])
  return block_df

In [12]:
import time
start = time.time()
pdf_path = '/content/drive/MyDrive/ Image CVs/cvSample4.pdf'
print(pdfimage2text(pdf_path))
print(time.time()-start)

        x      y  ...      h                                               text
0   139.0   97.0  ...  129.0                                     SƠ YẾU LY LỊCH
1    48.0  228.0  ...  254.0                                L/Thông tin cá nhân
2    47.0  255.0  ...  281.0                          Họ và tên: TRẦN NGỌC DIỄM
3    49.0  286.0  ...  310.0                                      Giới tính: Nữ
4    48.0  312.0  ...  338.0                                Quốc tịch: Việt Nam
..    ...    ...  ...    ...                                                ...
2    84.0  115.0  ...  142.0  bản thân nhằm đóng góp có hiệu quả vào sự phát...
3    79.0  140.0  ...  172.0          Mong muốn được gần bô lâu dài với công ty
4    69.0  175.0  ...  193.0                                                - M
5    81.0  172.0  ...  198.0  Môi trường làm việc thân thiện, năng động, chu...
6    85.0  191.0  ...  215.0                                        thăng tiến.

[142 rows x 5 columns]
22.5816674232482